# CNN classification

This code has been developed starting from the the online [Pytorch Full Course video](https://www.youtube.com/watch?v=V_xro1bcAuA&t=53736s). This is done for making some practice in developing models from scratch using `torch` and `torchvision` libraries.

In [ ]:
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from matplotlib import pyplot as plt

import os
from random import randint

## Dataset
For the example we'll use the `FashionMNIST` dataset directly available in `torchvision.datasets`.

### Load and visualize the dataset

In [ ]:
download = True if os.path.exists('Datasets/FashionMNIST') else False

# Load the training and testing dataset
train_dataset = datasets.FashionMNIST(download=True, root='Datasets', target_transform=None, transform=ToTensor(), train=True)
test_dataset = datasets.FashionMNIST(download=True, root='Datasets', target_transform=None, transform=ToTensor(), train=False)

# Print dataset infos
print(f'Dataset Infos:\n\n{train_dataset}\n\n{test_dataset}')
print(f'\nClasses: {train_dataset.class_to_idx}')
print(f'\nData shape ((CxHxW), class), e.g. ({train_dataset[0][0].shape}, {train_dataset[0][1]})')

# Print some examples
fig = plt.figure(figsize=(9,9))
rows, cols = 4,4
for i in range(1, rows*cols+1):
    index = randint(0, len(train_dataset))
    img, label = train_dataset[index]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(train_dataset.classes[label])
    plt.axis(False)

### Create a DataLoader
The dataloader is a python iterable, in this case we want to turn create batches of data. 

In [ ]:
from torch.utils.data import DataLoader
import yaml

with open('./configs.yml', 'r') as f:
    configs = yaml.safe_load(f)
    f.close()

print(f'Loaded configuration from configs.yml:\n{configs}')

train_dataloader = DataLoader(dataset=train_dataset, batch_size=configs['batch_size'], shuffle=True, num_workers=2)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=configs['batch_size'], shuffle=False, num_workers=2)

imgs_batch_example, labels_batch_example = next(iter(train_dataloader))
print(f'\nBatches shape (img_batches, labels_batches): ({imgs_batch_example.shape}, {labels_batch_example.shape})')

## Model

The implemented models are some simple CNN architectures available in `models.py` file.

### Import the model

In [ ]:
from models import FashionModelV0

output_shape = len(train_dataset.classes)
width, heigth = (imgs_batch_example.shape[2], imgs_batch_example.shape[3])
device = configs['device']

model = FashionModelV0(img_width=width, img_height=heigth, hidden_units=configs['hidden_units'], output_shape=output_shape).to(device)

### Setup the loss, optimizer and evaluation metrics

In [ ]:
import torch
from torch import nn

from evaluation import evaluate_model

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=configs['lr'])

### Create a training and testing loop

In [ ]:
from tqdm import tqdm

epochs = configs['epochs']
for epoch in tqdm(range(epochs)):
    print(f'\n\n----Starting epoch {epoch}----\n')

    train_loss = 0
    for batch_idx, (images, labels) in enumerate(train_dataloader):
        model.train()
        
        # Move data to the correct device
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        predictions = model(images) 
        
        # Loss calculation
        loss = loss_fn(predictions, labels)
        train_loss += loss

        # Optimizer zero grad
        optimizer.zero_grad()

        # Backpropagation
        loss.backward()

        # Optimizer optimization 
        optimizer.step()
    
    # Adjust the traing loss doing the average of the loss per epoch
    train_loss = train_loss/len(train_dataloader)
    
    # Testing
    test_results = evaluate_model(model, loss_fn, test_dataloader, device)
    print(test_results)
